#### synthBA inference for VS AIIMs data

In [1]:
import pandas as pd
import SimpleITK as sitk
import os

In [2]:
os.listdir('/home/omen/Documents/Nafisha/RandomForestData')

['masks',
 'rtfiles',
 'images',
 'dynunetFeat.csv',
 'radiomicsFeat.csv',
 'forsegInf.csv']

In [3]:
csv_path= '/home/omen/Documents/Nafisha/RandomForestData/forsegInf.csv'
df= pd.read_csv(csv_path)
df

,Unnamed: 0,PatientID,PatientsName,StudyInstanceUID,StudyDate,StudyDescription,Modality,SeriesDescription,image_path,rtfiles,mask_folder_path,SegmentationPath
0,0,100177883,Archna Archna Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.239178,20150903,MRI GK Schwannoma,MR,FIESTA,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
1,1,100177883,Archna Archna Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.1230760,20230404,MRI GK Schwannoma Follow Up,MR,Ax 3D T1+C,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
2,2,100535108,Krishna Master Mr,1.2.840.113619.6.95.31.0.3.4.1.6137.13.206762,20150528,MRI Brain Gamma Knife,MR,Ax FSPGR 3D 1mm,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
3,3,100535108,Krishna Master Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.1010219,20230919,MRI GK Others Follow Up,MR,Ax 3D T1+C,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
4,4,100793264,Mittal Naveen Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.191446,20150410,MRI GK Schwannoma,MR,AX T2 HR 2mm,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
...,...,...,...,...,...,...,...,...,...,...,...,...
70,70,104455962,Sharma Tribhuvan Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.998234,20230814,MRI GK Schwannoma Follow Up,MR,Ax 3D T1+C,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
71,71,104533436,Athar Muhammad Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.814921,20190725,MRI GK Schwannoma,MR,Ax 3D T1+C,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
72,72,104533436,Athar Muhammad Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.1013584,20230111,MRI GK Schwannoma Follow Up,MR,Ax 3D T1+C,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...
73,73,20130037201,Kumar Vipin Mr.,1.2.840.113619.6.95.31.0.3.4.1.6137.13.576871,20180102,MRI GK Schwannoma,MR,Ax 3D T1+C,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...,/home/omen/Documents/Nafisha/RandomForestData/...


In [27]:
import SimpleITK as sitk
import pandas as pd

new_data = []

reader = sitk.ImageSeriesReader()
reader.MetaDataDictionaryArrayUpdateOn()
reader.LoadPrivateTagsOn()

for _, row in df.iterrows():
    dicom_dir = row['image_path']
    output_path = dicom_dir.replace('images', 'image_nifti').rstrip('/') + '.nii.gz'

    # Read DICOM series
    dicom_names = reader.GetGDCMSeriesFileNames(dicom_dir)
    reader.SetFileNames(dicom_names)
    image3D = reader.Execute()

    # Extract metadata for the first slice
    patient_name = reader.GetMetaData(0, '0010|0010') if reader.HasMetaDataKey(0, '0010|0010') else ''
    birth_date = reader.GetMetaData(0, '0010|0030') if reader.HasMetaDataKey(0, '0010|0030') else ''
    acq_date = reader.GetMetaData(0, '0008|0022') if reader.HasMetaDataKey(0, '0008|0022') else ''

    if acq_date!='' and birth_date!='':
        # print(birth_date, acq_date, dicom_dir)
        age = int(acq_date[:4]) - int(birth_date[:4])
    else:
        age=''

    # Reorder axes and save to NIfTI
    image = sitk.PermuteAxes(image3D, [2, 1, 0])
    # sitk.WriteImage(image, output_path)

    new_data.append({
        "patient_name": patient_name,
        "Date of birth": birth_date,
        "Date of acquisition": acq_date,
        "Age": age,
        "path": output_path
    })

metadata_df = pd.DataFrame(new_data, columns=["patient_name", "Date of birth", "Date of acquisition", "Age", "path"])



In [28]:
metadata_df

,patient_name,Date of birth,Date of acquisition,Age,path
0,Archna^Archna^^Mr.,19750331,20150903,40,/home/omen/Documents/Nafisha/RandomForestData/...
1,Archna^Archna^^Mr.,19750331,20230404,48,/home/omen/Documents/Nafisha/RandomForestData/...
2,Krishna^Master^^Mr,19960915,20150528,19,/home/omen/Documents/Nafisha/RandomForestData/...
3,Krishna^Master^^Mr.,19960915,20230919,27,/home/omen/Documents/Nafisha/RandomForestData/...
4,Mittal^Naveen^^Mr.,19760210,20150410,39,/home/omen/Documents/Nafisha/RandomForestData/...
...,...,...,...,...,...
70,Sharma^Tribhuvan^^Mr.,19840425,20230814,39,/home/omen/Documents/Nafisha/RandomForestData/...
71,Athar^Muhammad^^Mr.,20051024,20190725,14,/home/omen/Documents/Nafisha/RandomForestData/...
72,Athar^Muhammad^^Mr.,20051024,20230111,18,/home/omen/Documents/Nafisha/RandomForestData/...
73,Kumar^Vipin^^Mr.,19500122,20180102,68,/home/omen/Documents/Nafisha/RandomForestData/...


In [29]:
metadata_df.to_csv('/home/omen/Documents/Nafisha/RandomForestData/forAgePred')